In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import numpy as np
import pandas as pd
import os
from os.path import expanduser
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
dataPath = "/content/drive/MyDrive/advanced ml/toxic/"

train = pd.read_csv(dataPath + 'tc_train.csv',index_col=0)
train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
# Create embedding index from file in .txt format. First line contains 
# dictionary size and embedding dim. Fields are space separated
def get_embeddings(file_name):
    embeddings_index = {}
    with open(file_name, encoding="utf8") as f:
        for line in f:
            values = line.rstrip().split(' ')
            if len(values) > 2:
                embeddings_index[values[0]] = np.asarray(values[1:], dtype="float32")
    return embeddings_index
embeddings_path = '/content/drive/MyDrive/advanced ml/toxic/'
embeddings_index = get_embeddings(embeddings_path+'crawl-300d-2M.vec')

In [ ]:
import string
trans_table = str.maketrans({key: ' ' for key in string.digits + '\r\n' +
                             string.punctuation.replace("\'",'')})
def preprocess(text):
    return ' '.join(text.lower().translate(trans_table).split(' '))

train['comment_clean'] = train.comment_text.apply(preprocess)
train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_clean
id,,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation why the edits made under my userna...
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww he matches this background colour i'm s...
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man i'm really not trying to edit war it...
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,more i can't make any real suggestions on im...
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,you sir are my hero any chance you remember...


In [ ]:
types = list(train)[1:]

from sklearn.feature_extraction.text import CountVectorizer

UNKNOWN_PROXY = 'unknown'
MIN_WORD_OCCURRENCE = 5

print("Creating the vocabulary of words occurred more than", MIN_WORD_OCCURRENCE)
vectorizer = CountVectorizer(lowercase=False, token_pattern="\S+", 
                             min_df=MIN_WORD_OCCURRENCE)
vectorizer.fit(train.comment_clean)

top_words = set(vectorizer.vocabulary_.keys())
top_words.add(UNKNOWN_PROXY)
print(len(top_words),'top words')

Creating the vocabulary of words occurred more than 5
45259 top words


In [ ]:
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

In [ ]:
tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(train.comment_clean)
print(tokenizer)

In [ ]:
word_index = tokenizer.word_index
print(len(word_index))
list(word_index.items())[:16]

280518


[('the', 1),
 ('to', 2),
 ('of', 3),
 ('and', 4),
 ('a', 5),
 ('you', 6),
 ('i', 7),
 ('is', 8),
 ('that', 9),
 ('in', 10),
 ('it', 11),
 ('for', 12),
 ('this', 13),
 ('not', 14),
 ('on', 15),
 ('be', 16)]

In [ ]:
MAX_SEQUENCE_LENGTH = 50
seq = tokenizer.texts_to_sequences(train.comment_clean)
data = pad_sequences(seq,maxlen=MAX_SEQUENCE_LENGTH,padding='post',
                     truncating='post')
with open(dataPath + 'comments.pkl','wb') as f: pickle.dump(data, f, -1)

print('\nFirst 2 sequences in `seq`: ',seq[:2])
print('\nShape of `data`: ',data.shape)
print('\nFirst prepared text in `data`:',data[0])


First 2 sequences in `seq`:  [[676, 77, 1, 133, 130, 177, 30, 666, 4436, 11406, 1126, 85, 349, 51, 2184, 12587, 50, 6354, 15, 59, 2567, 148, 7, 2795, 33, 116, 1196, 15967, 2453, 4, 47, 60, 247, 1, 359, 31, 1, 41, 27, 143, 71, 3503, 89], [121402, 52, 2765, 13, 466, 3656, 71, 4530, 2696, 21, 93, 41, 968, 196]]

Shape of `data`:  (223549, 50)

First prepared text in `data`: [  676    77     1   133   130   177    30   666  4436 11406  1126    85
   349    51  2184 12587    50  6354    15    59  2567   148     7  2795
    33   116  1196 15967  2453     4    47    60   247     1   359    31
     1    41    27   143    71  3503    89     0     0     0     0     0
     0     0]


In [ ]:
embeddings_dim = len(next(iter(embeddings_index.values())))
embeddings_dim

300

In [ ]:
# Create embedding matrix
def get_embedding_matrix(word_index,embeddings_index):
    nb_words = len(word_index) + 1 # +1 since min(word_index.values())=1
    embedding_matrix = np.zeros((nb_words,embeddings_dim))
    unknown = 0
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is None: unknown += 1
        else: embedding_matrix[i] = embedding_vector
    return embedding_matrix, unknown

In [ ]:
# Create embedding_layer and save it.
def make_save_emb_layer(word_index,embeddings_index,layer_file_name):
    embedding_matrix,unknown = get_embedding_matrix(word_index,embeddings_index)
    embedding_layer = Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],
                                weights=[embedding_matrix],trainable=False)
    with open(layer_file_name,'wb') as f: 
        pickle.dump(embedding_layer, f, -1)
    return unknown

EMBEDDING_LAYER_FILE = dataPath + 'toxic_comments_embed_layer.pkl'
print(make_save_emb_layer(word_index,embeddings_index,EMBEDDING_LAYER_FILE),
      'unknown words')

160113 unknown words


In [ ]:
import pickle
from sklearn.model_selection import train_test_split

# convert each vector of labels to the string
labels = train[types].astype(str).apply(lambda x: ''.join(x),axis=1)
print('Labels: \n',labels.head())
# aggregate rare combinations if any
count = labels.value_counts()
rare = count.index[count<=2]
labels[np.isin(labels.values,rare)] = 'rare'
print('\nCounts of labels: \n',labels.value_counts())
train_index, val_index = train_test_split(list(range(data.shape[0])), test_size=0.2, 
                                      stratify = labels, random_state=0)
# save train and validation indices for further calculations
fname = dataPath + 'train_val_split.pkl'
with open(fname, 'wb') as f: pickle.dump([train_index, val_index], f, -1)

Labels: 
 id
0000997932d777bf    000000explanation why the edits made under my ...
000103f0d9cfb60f    000000d'aww  he matches this background colour...
000113f07ec002fd    000000hey man  i'm really not trying to edit w...
0001b41b1c6bb37e    000000  more i can't make any real suggestions...
0001d958c54c6e35    000000you  sir  are my hero  any chance you re...
dtype: object

Counts of labels: 
 rare                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
from tensorflow.keras.layers import Dense,Embedding,Input,Dropout,Conv1D
from tensorflow.keras.layers import SpatialDropout1D, Flatten,LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from sklearn.metrics import roc_auc_score
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate

def get_model():
    input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,))
    x = embedding_layer(input_layer)
    x = SpatialDropout1D(0.5)(x)
    x = LSTM(10, return_sequences=True)(x)
    x = Conv1D(5, kernel_size=2, padding="valid")(x)
    max = GlobalMaxPooling1D()(x)
    avg = GlobalAveragePooling1D()(x)
    x = concatenate([max,avg])
    x = BatchNormalization()(x)
    x = Dropout(.2)(x)
    output_layer = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss="binary_crossentropy", optimizer=Adam())
    return model

In [ ]:
with open(EMBEDDING_LAYER_FILE, 'rb') as f: embedding_layer = pickle.load(f)
with open(dataPath + 'comments.pkl', 'rb') as f: data = pickle.load(f)   

X_train = data[train_index]
X_val = data[val_index]

y_train = train.iloc[train_index][['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
y_val = train.iloc[val_index][['toxic','severe_toxic','obscene','threat','insult','identity_hate']]

In [ ]:
best_model_path = 'best_model.h5'
BATCH_SIZE = 512

early_stopping = EarlyStopping(patience=5)
model_checkpoint = ModelCheckpoint(best_model_path,
                                   save_best_only=True, save_weights_only=True)

model = get_model()
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 300)      84155700    ['input_1[0][0]']                
                                                                                                  
 spatial_dropout1d (SpatialDrop  (None, 50, 300)     0           ['embedding[0][0]']              
 out1D)                                                                                           
                                                                                                  
 lstm (LSTM)                    (None, 50, 10)       12440       ['spatial_dropout1d[0][0]']  

In [ ]:
hist = model.fit(X_train, y_train,validation_data=(X_val, y_val),
                 epochs=28, batch_size=BATCH_SIZE, shuffle=True, verbose=2,
                 callbacks=[model_checkpoint, early_stopping])
model.load_weights(best_model_path)
val_pred = model.predict(X_val, batch_size=BATCH_SIZE, verbose=0)
print('validation AUC',roc_auc_score(y_val, val_pred))

Epoch 1/28
350/350 - 48s - loss: 0.0565 - val_loss: 0.0510 - 48s/epoch - 136ms/step
Epoch 2/28
350/350 - 48s - loss: 0.0566 - val_loss: 0.0509 - 48s/epoch - 137ms/step
Epoch 3/28
350/350 - 52s - loss: 0.0564 - val_loss: 0.0506 - 52s/epoch - 147ms/step
Epoch 4/28
350/350 - 49s - loss: 0.0565 - val_loss: 0.0506 - 49s/epoch - 139ms/step
Epoch 5/28
350/350 - 50s - loss: 0.0562 - val_loss: 0.0506 - 50s/epoch - 143ms/step
Epoch 6/28
350/350 - 49s - loss: 0.0561 - val_loss: 0.0517 - 49s/epoch - 139ms/step
Epoch 7/28
350/350 - 50s - loss: 0.0559 - val_loss: 0.0507 - 50s/epoch - 142ms/step
Epoch 8/28
350/350 - 51s - loss: 0.0559 - val_loss: 0.0508 - 51s/epoch - 145ms/step
Epoch 9/28
350/350 - 52s - loss: 0.0553 - val_loss: 0.0505 - 52s/epoch - 147ms/step
Epoch 10/28
350/350 - 51s - loss: 0.0556 - val_loss: 0.0502 - 51s/epoch - 145ms/step
Epoch 11/28
350/350 - 50s - loss: 0.0554 - val_loss: 0.0509 - 50s/epoch - 142ms/step
Epoch 12/28
350/350 - 54s - loss: 0.0554 - val_loss: 0.0503 - 54s/epoch - 

In [ ]:
test = pd.read_csv(dataPath + 'tc_test.csv',index_col=0)

test['comment_clean'] = test.comment_text.apply(preprocess)

test_seq = tokenizer.texts_to_sequences(test.comment_clean)
test_data = pad_sequences(test_seq,maxlen=MAX_SEQUENCE_LENGTH,padding='post',
                         truncating='post')
with open(dataPath + 'test_comments.pkl','wb') as f: pickle.dump(data, f, -1)
test_pred = model.predict(test_data, batch_size=BATCH_SIZE, verbose=0)
out = pd.DataFrame(test_pred, index=test.index)
out.columns = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
out.to_csv(dataPath + 'tc_output.csv')